Empty properties and second homes

In [1]:
from gssutils import *

scraper = Scraper('https://www2.gov.scot/Topics/Statistics/Browse/Housing-Regeneration/HSfS/LTemptysecondhomes/EmptySecondhometables')
scraper

## Empty properties and second homes

### Description

Information on empty properties and second homes is collected each year from
Local Authorities through the CTAXBASE data collection. Empty properties are
of particular interest as they can help increase the supply of housing in
Scotland when brought back into use.

Data presented is for:

  * **Unoccupied Exemptions:** generally properties which are empty and unfurnished for less than 6 months and exempt from paying council tax.

  * **Long Term Empty Properties:** properties which have been empty for more than 6 months and are liable for council tax.

  * **Second Homes:** homes which are furnished and lived in for at least 25 days in a 12 month period but not as someone’s main residence.



### Distributions

1. Long term empty and second home web tables  ([application/vnd.openxmlformats-officedocument.spreadsheetml.sheet](https://www2.gov.scot/Resource/0054/00543818.xlsx))


In [2]:
tabs = {tab.name: tab for tab in scraper.distribution().as_databaker()}
tabs.keys()

dict_keys(['Contents', 'Second Homes & Empty properties', 'Unoccupied exemptions', 'Summary table', 'Long term empty discount'])

In [3]:
tabs = scraper.distribution().as_databaker()

In [4]:
tab = next(t for t in tabs if t.name=='Second Homes & Empty properties')
cell = tab.filter(contains_string('2005'))
year = tab.filter(contains_string('Scotland')).shift(0,-1).fill(RIGHT)\
        .is_not_blank().is_not_whitespace()
observations = year.fill(DOWN).is_not_blank().is_not_whitespace().is_number() - year
area = cell.fill(DOWN).is_not_blank().is_not_whitespace() - cell

In [5]:
Dimensions = [
            HDim(year,'Year',DIRECTLY,ABOVE),
            HDim(area,'Geography',DIRECTLY,LEFT),
            HDim(cell,'Tenure',CLOSEST,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit', 'dwellings')
            ]
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
table = c1.topandas()

In [6]:
table['Tenure'] = table['Tenure'].map(
    lambda x: {
        'Second Homes and long term empty1 properties, 2005 - 2018' : 'second-homes-and-long-term-empty-properties/total',
       'Long term empty1 properties, 2005 - 2018' : 'second-homes-and-long-term-empty-properties/long-term-empty-properties',
       'Second Homes, 2005 - 2018' : 'second-homes-and-long-term-empty-properties/second-homes'
        }.get(x, x))

In [7]:
table['Period'] = 'year/' + table['Year'].astype(str).str[0:4]

In [8]:
import numpy as np
table['OBS'].replace('', np.nan, inplace=True)
table.dropna(subset=['OBS'], inplace=True)
table.rename(columns={'OBS': 'Value'}, inplace=True)
table['Value'] = table['Value'].astype(int)

In [9]:
table['Geography'] = table['Geography'].str.rstrip('12345678')
table['Geography'] = table['Geography'].str.strip()

In [10]:
scotland_gss_codes = pd.read_csv('scotland-gss.csv', index_col='Area')
table['Geography'] = table['Geography'].map(
    lambda x: scotland_gss_codes.loc[x]['Code']
)

In [11]:
table = table[['Period','Geography','Tenure','Measure Type','Value','Unit']]

In [12]:
tidy = pd.DataFrame()
tidy = pd.concat([tidy , table])

In [13]:
tab1 = next(t for t in tabs if t.name=='Unoccupied exemptions')
year1 = tab1.filter(contains_string('SCOTLAND')).shift(0,-1).fill(RIGHT).is_not_blank().is_not_whitespace()
observations1 = year1.fill(DOWN).is_not_blank().is_not_whitespace().is_number() - year1
area1 = tab1.filter(contains_string('SCOTLAND')).expand(DOWN).is_not_blank().is_not_whitespace()

In [14]:
Dimensions1 = [
            HDim(year1,'Year',DIRECTLY,ABOVE),
            HDim(area1,'Geography',DIRECTLY,LEFT),
            HDimConst('Tenure','second-homes-and-long-term-empty-properties/unoccupied-exemptions'),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit', 'dwellings')
            ]
c2 = ConversionSegment(observations1, Dimensions1, processTIMEUNIT=True)
table1 = c2.topandas()

In [15]:
table1['Period'] = 'year/' + table1['Year'].astype(str).str[0:4]

In [16]:
import numpy as np
table1['OBS'].replace('', np.nan, inplace=True)
table1.dropna(subset=['OBS'], inplace=True)
table1.rename(columns={'OBS': 'Value'}, inplace=True)
table1['Value'] = table1['Value'].astype(int)

In [17]:
table1['Geography'] = table1['Geography'].map(
    lambda x: {
        'SCOTLAND' : 'Scotland',
        'Glasgow' : 'Glasgow City'
        }.get(x, x))


In [18]:
scotland_gss_codes = pd.read_csv('scotland-gss.csv', index_col='Area')
table1['Geography'] = table1['Geography'].map(
    lambda x: scotland_gss_codes.loc[x]['Code']
)

In [19]:
table1 = table1[['Period','Geography','Tenure','Measure Type','Value','Unit']]

In [20]:
tidy = pd.concat([tidy , table1])

In [21]:
out = Path('out')
out.mkdir(exist_ok=True)
tidy.drop_duplicates().to_csv(out / 'observations.csv', index = False)

In [22]:
scraper.dataset.family = 'housing'
scraper.dataset.theme = THEME['housing-planning-local-services']
with open(out / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())
    
schema = CSVWSchema('https://ons-opendata.github.io/ref_housing/')
schema.create(out / 'observations.csv', out / 'observations.csv-schema.json')

In [23]:
tidy

,Period,Geography,Tenure,Measure Type,Value,Unit
0,year/2005,S92000003,second-homes-and-long-term-empty-properties/total,Count,55155,dwellings
1,year/2006,S92000003,second-homes-and-long-term-empty-properties/total,Count,52823,dwellings
2,year/2007,S92000003,second-homes-and-long-term-empty-properties/total,Count,55385,dwellings
3,year/2008,S92000003,second-homes-and-long-term-empty-properties/total,Count,57820,dwellings
4,year/2009,S92000003,second-homes-and-long-term-empty-properties/total,Count,59229,dwellings
5,year/2010,S92000003,second-homes-and-long-term-empty-properties/total,Count,62600,dwellings
6,year/2011,S92000003,second-homes-and-long-term-empty-properties/total,Count,64606,dwellings
7,year/2012,S92000003,second-homes-and-long-term-empty-properties/total,Count,66053,dwellings
8,year/2013,S92000003,second-homes-and-long-term-empty-properties/total,Count,63061,dwellings
9,year/2014,S92000003,second-homes-and-long-term-empty-properties/total,Count,59763,dwellings
